# Notes
- find_dotenv() searches for a .env file in your current directory and parent directories.
- load_dotenv() reads the .env file and loads any variables (like API keys) into your environment, so you can access them with os.environ["VARIABLE_NAME"].
-  Setting verbose=True enables detailed logging of the internal operations and steps taken by the chain during execution. This means that when you run the conversation, you will see additional output in your console or notebook, such as the prompts sent to the language model, the responses received, and any intermediate processing steps.
- Using verbose=True is especially helpful for debugging and understanding how the chain processes inputs and generates outputs. It allows you to trace the flow of data and see exactly what is happening behind the scenes, which can be valuable when developing or troubleshooting conversational AI applications. If you prefer a cleaner output without these details, you can set verbose=False or simply omit the parameter, as it defaults to False.
- You can see conversastion history with verbose=True
- ConversationBufferMemory is how u store memory but u can also add to this memory
- llms are stateless ie. dont have memory by default. the conversation is typically passed as context to use when responding. you have to think about how expensive this can be as the chat grows, so u may want to choose other methods provided by langchain like ConversationBufferWindowMemory 
- ConversationBufferWindowMemory only stores a window of memory that u can specify
- ConversationTokenBufferMemory stores certain amm tokens. since most llm pricing is about tokens, this maps more directly to cost
- ConversationSummaryMemory summarises convo & makes that the memory
- also vector data memory which stores text in vector db & retrieves mosst relevant block of text
- Entity memory remembers details about specific entities
- you can combine diff types of memory eg entity + conversaion
- can store in normal db either

In [72]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) #read local .env file

## ConversationBufferMemory

In [73]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

In [74]:
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

In [75]:
conversation.predict(input="hi my names nicole")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hi my names nicole
AI:

> Finished chain.


"Hello Nicole! It's nice to meet you. How can I assist you today?"

In [76]:
conversation.predict(input="im 25")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi my names nicole
AI: Hello Nicole! It's nice to meet you. How can I assist you today?
Human: im 25
AI:

> Finished chain.


"That's great to know, Nicole! Being 25 is a wonderful age with so many opportunities ahead. Is there anything specific you'd like to know or talk about today?"

In [77]:
print(memory.buffer)

Human: hi my names nicole
AI: Hello Nicole! It's nice to meet you. How can I assist you today?
Human: im 25
AI: That's great to know, Nicole! Being 25 is a wonderful age with so many opportunities ahead. Is there anything specific you'd like to know or talk about today?


In [78]:
#access everything langchain has stored
memory.load_memory_variables({})

{'history': "Human: hi my names nicole\nAI: Hello Nicole! It's nice to meet you. How can I assist you today?\nHuman: im 25\nAI: That's great to know, Nicole! Being 25 is a wonderful age with so many opportunities ahead. Is there anything specific you'd like to know or talk about today?"}

In [79]:
# add to memory (appends to end)
memory.save_context(
    {"input": "Hi"}, {"output": "hi, how are you!"}
)
print(memory.buffer)

Human: hi my names nicole
AI: Hello Nicole! It's nice to meet you. How can I assist you today?
Human: im 25
AI: That's great to know, Nicole! Being 25 is a wonderful age with so many opportunities ahead. Is there anything specific you'd like to know or talk about today?
Human: Hi
AI: hi, how are you!


## ConversationBufferWindowMemory

In [80]:
from langchain.memory import ConversationBufferWindowMemory

In [81]:
# k: Use to keep track of the last k turns of a conversation.
# If the number of messages in the conversation is more than the maximum number of messages to keep, the oldest messages are dropped.
# k=1 means 1 convo exchang ie 1 question & its ans
memory = ConversationBufferWindowMemory(k=1)

In [82]:
# add to memory (appends to end)
# notice how it only keeps the last
memory.save_context(
    {"input": "Hi"}, {"output": "hi, how are you!"}
)
memory.save_context(
    {"input": "whats the weather"}, {"output": "theres currently a heat wave"}
)
print(memory.buffer)

Human: whats the weather
AI: theres currently a heat wave


In [83]:
llm = ChatOpenAI(temperature=0.0)
memory=ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm,
    memory=memory,
    verbose=True
)

In [84]:
conversation.predict(input="hi my name is nicole")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: hi my name is nicole
AI:

> Finished chain.


"Hello Nicole! It's nice to meet you. How can I assist you today?"

In [85]:
conversation.predict(input="whats 1+1")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: hi my name is nicole
AI: Hello Nicole! It's nice to meet you. How can I assist you today?
Human: whats 1+1
AI:

> Finished chain.


'1 + 1 equals 2. Is there anything else you would like to know?'

In [86]:
# wont remember this since k=1
conversation.predict(input="whats my name")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: whats 1+1
AI: 1 + 1 equals 2. Is there anything else you would like to know?
Human: whats my name
AI:

> Finished chain.


"I'm sorry, I do not have access to personal information such as your name. Is there anything else you would like to know?"

## ConversationTokenBufferMemory

In [87]:
#!pip install tiktoken

In [88]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm= ChatOpenAI(temperature=0.0)

In [89]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=10)
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [90]:
memory.load_memory_variables({})
print(memory.buffer)

In [91]:
memory.max_token_limit=2
memory.load_memory_variables({})
print(memory.buffer)

## ConversationSummaryMemory

In [92]:
from langchain.memory import ConversationBufferMemory

In [93]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory2 = ConversationBufferMemory(llm=llm, max_token_limit=5)
memory2.save_context({"input": "Hello"}, {"output": "What's up"})
memory2.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory2.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [94]:
memory2.load_memory_variables({})

{'history': "Human: Hello\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool\nHuman: What is on the schedule today?\nAI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo."}

In [95]:
conversation = ConversationChain(
    llm=llm,
    memory=memory2,
    verbose=True
)

In [96]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hello
AI: What's up
Human: Not much, just hanging
AI: Cool
Human: What is on the schedule today?
AI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.
Human: What would be a good demo to show?
AI:

> Finished chain.


'I would recommend showcasing the latest Language Model demo, as it is a cutting-edge technology that your customer will likely be very interested in. It demonstrates the power and capabilities of our AI technology in a tangible way. Additionally, you could also show some real-world examples of how our AI has helped other customers improve their business processes and outcomes.'

In [97]:
memory2.load_memory_variables({})

{'history': "Human: Hello\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool\nHuman: What is on the schedule today?\nAI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.\nHuman: What would be a good demo to show?\nAI: I would recommend showcasing the latest Language Model demo, as it is a cutting-edge technology that your customer will likely be very interested in. It demonstrates the power and capabilities of our AI technology in a tangible way. Additionally, you could also show some real-world examples of how our AI has helped other customers improve their business processes and outcomes."}